In [1]:
import os
import pickle

from gensim.models import KeyedVectors
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, HalvingGridSearchCV
from sklearn.neural_network import MLPClassifier

from data import preprocess_data, vectorize_data, load_dataset
from evaluation import analysis, evaluate_models_with_data
from w2v_adapter import Word2VecAdapter

import advanced_processor_chain_factory
import simple_processor_chain_factory

In [2]:
dataset = load_dataset()
DEBUG = True

# Inspection of Pre-Processing Approaches

In [3]:
models = {'logistic regression' : LogisticRegression(class_weight = 'balanced', n_jobs=-1),
          'svm' : svm.LinearSVC(),
          'knn' : KNeighborsClassifier(n_neighbors=8, n_jobs=-1)
         }

## Without Pre-Process

In [4]:
evaluate_models_with_data(models, *vectorize_data(*preprocess_data(dataset, debug=DEBUG), CountVectorizer(max_features=2000)))

------Evaluating logistic regression------
Report: Classification
               precision    recall  f1-score   support

    positive       0.84      0.85      0.84       489
    negative       0.85      0.85      0.85       511

    accuracy                           0.85      1000
   macro avg       0.85      0.85      0.85      1000
weighted avg       0.85      0.85      0.85      1000

Matrix: Confusion
 [[415  74]
 [ 79 432]]
Accuracy:
 0.847
------Evaluating svm------


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Report: Classification
               precision    recall  f1-score   support

    positive       0.82      0.84      0.83       489
    negative       0.85      0.82      0.83       511

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

Matrix: Confusion
 [[412  77]
 [ 91 420]]
Accuracy:
 0.832
------Evaluating knn------
Report: Classification
               precision    recall  f1-score   support

    positive       0.61      0.65      0.63       489
    negative       0.64      0.60      0.62       511

    accuracy                           0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

Matrix: Confusion
 [[320 169]
 [205 306]]
Accuracy:
 0.626


## Simple Pre-Process

In [5]:
evaluate_models_with_data(models,
                          *vectorize_data(
                              *preprocess_data(dataset, processor_chain=simple_processor_chain_factory.create(), debug=DEBUG),
                              CountVectorizer(max_features=2000)))

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

------Evaluating logistic regression------
Report: Classification
               precision    recall  f1-score   support

    positive       0.83      0.83      0.83       494
    negative       0.84      0.83      0.83       506

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

Matrix: Confusion
 [[411  83]
 [ 84 422]]
Accuracy:
 0.833
------Evaluating svm------


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Report: Classification
               precision    recall  f1-score   support

    positive       0.80      0.81      0.81       494
    negative       0.81      0.81      0.81       506

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000

Matrix: Confusion
 [[400  94]
 [ 98 408]]
Accuracy:
 0.808
------Evaluating knn------
Report: Classification
               precision    recall  f1-score   support

    positive       0.61      0.61      0.61       494
    negative       0.62      0.62      0.62       506

    accuracy                           0.61      1000
   macro avg       0.61      0.61      0.61      1000
weighted avg       0.61      0.61      0.61      1000

Matrix: Confusion
 [[300 194]
 [192 314]]
Accuracy:
 0.614


## Pre-Process with Stemmimg

In [6]:
evaluate_models_with_data(models,
                          *vectorize_data(
                              *preprocess_data(dataset, processor_chain=advanced_processor_chain_factory.create('stem'),
                                               debug=DEBUG),
                              CountVectorizer(max_features=2000)))

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

------Evaluating logistic regression------
Report: Classification
               precision    recall  f1-score   support

    positive       0.82      0.80      0.81       499
    negative       0.81      0.83      0.82       501

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000

Matrix: Confusion
 [[399 100]
 [ 87 414]]
Accuracy:
 0.813
------Evaluating svm------
Report: Classification
               precision    recall  f1-score   support

    positive       0.79      0.79      0.79       499
    negative       0.79      0.78      0.79       501

    accuracy                           0.79      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.79      0.79      0.79      1000

Matrix: Confusion
 [[396 103]
 [108 393]]
Accuracy:
 0.789
------Evaluating knn------
Report: Classification
               precision    recall  f1-score   support

    

## Pre-Process with Lemmitization

In [7]:
X, Y = preprocess_data(dataset, processor_chain=advanced_processor_chain_factory.create('lem'), debug=DEBUG)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
del dataset

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
vectorizer = CountVectorizer(max_features=2000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
evaluate_models_with_data(models, X_train_bow, X_test_bow, Y_train, Y_test)

------Evaluating logistic regression------
Report: Classification
               precision    recall  f1-score   support

    positive       0.85      0.80      0.82       519
    negative       0.80      0.84      0.82       481

    accuracy                           0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000

Matrix: Confusion
 [[417 102]
 [ 75 406]]
Accuracy:
 0.823
------Evaluating svm------
Report: Classification
               precision    recall  f1-score   support

    positive       0.82      0.77      0.79       519
    negative       0.77      0.81      0.79       481

    accuracy                           0.79      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.79      0.79      0.79      1000

Matrix: Confusion
 [[401 118]
 [ 90 391]]
Accuracy:
 0.792
------Evaluating knn------
Report: Classification
               precision    recall  f1-score   support

    

In [9]:
del models

# Compare W2V and BoW with Their Best Tuned Hyper-parameters

In [10]:
kfold = StratifiedKFold(n_splits=5)
general_grid_params = {'verbose' : 1, 'cv' : kfold, 'n_jobs' : -1, 'scoring' : 'f1'}

logistic_grid = {
    'penalty':['l2'],
    'C':[1, 300, 500, 700, 900, 2000],
    'class_weight':['balanced'],
    'solver':['saga'],
    'n_jobs':[-1],
    'max_iter':[1000],
}

svc_grid = {
    'kernel' : ['linear', 'rbf'],
    'C':[0.1, 1, 500, 1000],
}

knn_grid = {
    'n_neighbors' : [i for i in range(1,24,2)],
    'n_jobs' : [-1]
}

## BoW

### Logistic Regression

In [11]:
bow_log = LogisticRegression()
bow_log = GridSearchCV(estimator=bow_log, param_grid=logistic_grid, **general_grid_params)
bow_log.fit(X_train_bow, Y_train)
print(f'Best Score: {bow_log.best_score_}')
print(f'Best Params: {bow_log.best_params_}')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Score: 0.826463153420435
Best Params: {'C': 1, 'class_weight': 'balanced', 'max_iter': 1000, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'saga'}


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [12]:
analysis(Y_test, bow_log.predict(X_test_bow))

Report: Classification
               precision    recall  f1-score   support

    positive       0.86      0.81      0.84       519
    negative       0.81      0.86      0.83       481

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000

Matrix: Confusion
 [[422  97]
 [ 67 414]]
Accuracy:
 0.836


0.8346774193548387

### SVM

In [13]:
bow_svm = svm.SVC()
bow_svm = HalvingGridSearchCV(estimator=bow_svm, param_grid=svc_grid, cv = 4, n_jobs= -1, scoring='f1', factor=2)
bow_svm.fit(X_train_bow, Y_train)
print(f'Best Score: {bow_svm.best_score_}')
print(f'Best Params: {bow_svm.best_params_}')

Best Score: 0.8277515348400367
Best Params: {'C': 1000, 'kernel': 'rbf'}


In [14]:
analysis(Y_test, bow_svm.predict(X_test_bow))

Report: Classification
               precision    recall  f1-score   support

    positive       0.86      0.80      0.83       519
    negative       0.80      0.86      0.83       481

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

Matrix: Confusion
 [[416 103]
 [ 67 414]]
Accuracy:
 0.83


0.8296593186372746

### KNN

In [15]:
bow_knn = KNeighborsClassifier()
bow_knn = GridSearchCV(estimator=bow_knn, param_grid=knn_grid, **general_grid_params)
bow_knn.fit(X_train_bow, Y_train)
print(f'Best Score: {bow_svm.best_score_}')
print(f'Best Params: {bow_svm.best_params_}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Score: 0.8277515348400367
Best Params: {'C': 1000, 'kernel': 'rbf'}


In [16]:
analysis(Y_test, bow_svm.predict(X_test_bow))


Report: Classification
               precision    recall  f1-score   support

    positive       0.86      0.80      0.83       519
    negative       0.80      0.86      0.83       481

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

Matrix: Confusion
 [[416 103]
 [ 67 414]]
Accuracy:
 0.83


0.8296593186372746

## W2V

In [17]:
if os.path.isfile('w2v.kv'):
    vectorizer = Word2VecAdapter(pre_trained_model=KeyedVectors.load('w2v.kv'))
else:
    vectorizer = Word2VecAdapter()


X_train_w2v = vectorizer.fit_transform(X_train)
X_test_w2v = vectorizer.transform(X_test)

if not os.path.isfile('w2v.kv'):
     vectorizer.wv.save('w2v.kv')

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

### Logistic Regression

In [18]:
w2v_log = LogisticRegression(n_jobs=-1)
w2v_log.fit(X_train_w2v, Y_train)
analysis(Y_test, w2v_log.predict(X_test_w2v))

Report: Classification
               precision    recall  f1-score   support

    positive       0.90      0.85      0.87       519
    negative       0.85      0.90      0.87       481

    accuracy                           0.87      1000
   macro avg       0.87      0.87      0.87      1000
weighted avg       0.88      0.87      0.87      1000

Matrix: Confusion
 [[441  78]
 [ 48 433]]
Accuracy:
 0.874


0.8729838709677421

In [20]:
w2v_log = LogisticRegression()
w2v_log = GridSearchCV(estimator=w2v_log, param_grid=logistic_grid, **general_grid_params)
w2v_log.fit(X_train_w2v, Y_train)
print(f'Best Score: {w2v_log.best_score_}')
print(f'Best Params: {w2v_log.best_params_}')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best Score: 0.8591242828594753
Best Params: {'C': 500, 'class_weight': 'balanced', 'max_iter': 1000, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'saga'}


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [21]:
analysis(Y_test, w2v_log.predict(X_test_w2v))

Report: Classification
               precision    recall  f1-score   support

    positive       0.89      0.86      0.87       519
    negative       0.85      0.89      0.87       481

    accuracy                           0.87      1000
   macro avg       0.87      0.87      0.87      1000
weighted avg       0.87      0.87      0.87      1000

Matrix: Confusion
 [[444  75]
 [ 53 428]]
Accuracy:
 0.872


0.8699186991869918

### SVM

In [22]:
w2v_svm = svm.SVC()
w2v_svm = HalvingGridSearchCV(estimator=w2v_svm, param_grid=svc_grid, cv = 4, n_jobs= -1, scoring='f1', factor=2)
w2v_svm.fit(X_train_w2v, Y_train)
print(f'Best Score: {w2v_svm.best_score_}')
print(f'Best Params: {w2v_svm.best_params_}')

Best Score: 0.8608625791792022
Best Params: {'C': 1, 'kernel': 'rbf'}


In [23]:
analysis(Y_test, w2v_svm.predict(X_test_w2v))

Report: Classification
               precision    recall  f1-score   support

    positive       0.91      0.86      0.88       519
    negative       0.86      0.91      0.88       481

    accuracy                           0.88      1000
   macro avg       0.88      0.89      0.88      1000
weighted avg       0.89      0.88      0.88      1000

Matrix: Confusion
 [[445  74]
 [ 42 439]]
Accuracy:
 0.884


0.8832997987927566

### KNN

In [24]:
w2v_knn = KNeighborsClassifier()
w2v_knn = GridSearchCV(estimator=w2v_knn, param_grid=knn_grid, **general_grid_params)
w2v_knn.fit(X_train_w2v, Y_train)
print(f'Best Score: {w2v_svm.best_score_}')
print(f'Best Params: {w2v_svm.best_params_}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Score: 0.8608625791792022
Best Params: {'C': 1, 'kernel': 'rbf'}


In [25]:
analysis(Y_test, w2v_svm.predict(X_test_w2v))

Report: Classification
               precision    recall  f1-score   support

    positive       0.91      0.86      0.88       519
    negative       0.86      0.91      0.88       481

    accuracy                           0.88      1000
   macro avg       0.88      0.89      0.88      1000
weighted avg       0.89      0.88      0.88      1000

Matrix: Confusion
 [[445  74]
 [ 42 439]]
Accuracy:
 0.884


0.8832997987927566

## Comparison

In [26]:
summary = {
    'LR': {'BoW': bow_log, 'W2V': w2v_log},
    'SVM' : {'BoW': bow_svm, 'W2V': w2v_svm},
    'KNN': {'BoW': bow_knn, 'W2V': w2v_knn},
  }

for name, values in summary.items():
    print(f'For classifier {name}, best BoW score is {values["BoW"].best_score_}, whereas best W2V score is {values["W2V"].best_score_}')
    best_model = "BoW" if values["BoW"].best_score_ > values["W2V"].best_score_ else "W2V"
    print(f'So {best_model} is better with parameters {values[best_model].best_params_}')
    filename = name + '.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(values[best_model], f)

del summary, bow_log, w2v_log, bow_svm, w2v_svm, bow_knn, w2v_knn

For classifier LR, best BoW score is 0.826463153420435, whereas best W2V score is 0.8591242828594753
So W2V is better with parameters {'C': 500, 'class_weight': 'balanced', 'max_iter': 1000, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'saga'}
For classifier SVM, best BoW score is 0.8277515348400367, whereas best W2V score is 0.8608625791792022
So W2V is better with parameters {'C': 1, 'kernel': 'rbf'}
For classifier KNN, best BoW score is 0.6982935519972597, whereas best W2V score is 0.7921232157219126
So W2V is better with parameters {'n_jobs': -1, 'n_neighbors': 15}


# MLP

In [27]:
mlp_grid = {
    'hidden_layer_sizes':[(500, 250), (1000, 250), (500, 250, 250),(1000, 500,250)],
    'activation':['tanh', 'relu']
}
def eval_mlp(X_train, X_test, Y_train, Y_test):
    best_f1 = -1
    best_model = None
    for sizes in mlp_grid['hidden_layer_sizes']:
        for act in mlp_grid['activation']:
            m = MLPClassifier(hidden_layer_sizes=sizes, activation=act, solver='sgd', alpha=1,
                                    learning_rate='adaptive', max_iter=10)
            m.fit(X_train, Y_train)
            print(f'Model config: hidden_layer_sizes={sizes}, activation={act}')
            f1 = analysis(Y_test, m.predict(X_test))
            if f1 > best_f1:
                best_model = m
                best_f1 = f1
    return best_f1, best_model

## W2V

In [28]:
w2v_f1, w2v_mlp = eval_mlp(X_train_w2v, X_test_w2v, Y_train, Y_test)

/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.85      0.81      0.83       519
    negative       0.80      0.85      0.83       481

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

Matrix: Confusion
 [[420  99]
 [ 73 408]]
Accuracy:
 0.828


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.80      0.83      0.82       519
    negative       0.81      0.77      0.79       481

    accuracy                           0.80      1000
   macro avg       0.80      0.80      0.80      1000
weighted avg       0.80      0.80      0.80      1000

Matrix: Confusion
 [[432  87]
 [109 372]]
Accuracy:
 0.804


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.76      0.86      0.80       519
    negative       0.82      0.70      0.76       481

    accuracy                           0.78      1000
   macro avg       0.79      0.78      0.78      1000
weighted avg       0.79      0.78      0.78      1000

Matrix: Confusion
 [[447  72]
 [145 336]]
Accuracy:
 0.783


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(1000, 500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.86      0.82      0.84       519
    negative       0.82      0.85      0.84       481

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000

Matrix: Confusion
 [[427  92]
 [ 70 411]]
Accuracy:
 0.838
Model config: hidden_layer_sizes=(1000, 500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.79      0.76      0.78       519
    negative       0.75      0.78      0.77       481

    accuracy                           0.77      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.77      0.77      0.77      1000

Matrix: Confusion
 [[397 122]
 [107 374]]
Accuracy:
 0.771


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## BoW

In [29]:
bow_f1, bow_mlp = eval_mlp(X_train_bow, X_test_bow, Y_train, Y_test)

/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.81      0.72      0.76       519
    negative       0.73      0.81      0.77       481

    accuracy                           0.77      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.77      0.77      0.77      1000

Matrix: Confusion
 [[374 145]
 [ 89 392]]
Accuracy:
 0.766


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.75      0.56      0.64       519
    negative       0.63      0.80      0.70       481

    accuracy                           0.68      1000
   macro avg       0.69      0.68      0.67      1000
weighted avg       0.69      0.68      0.67      1000

Matrix: Confusion
 [[291 228]
 [ 97 384]]
Accuracy:
 0.675


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

Model config: hidden_layer_sizes=(500, 250, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.68      0.62      0.65       519
    negative       0.62      0.68      0.65       481

    accuracy                           0.65      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.65      0.65      0.65      1000

Matrix: Confusion
 [[322 197]
 [154 327]]
Accuracy:
 0.649


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(1000, 500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.84      0.75      0.79       519
    negative       0.76      0.84      0.80       481

    accuracy                           0.80      1000
   macro avg       0.80      0.80      0.80      1000
weighted avg       0.80      0.80      0.80      1000

Matrix: Confusion
 [[391 128]
 [ 75 406]]
Accuracy:
 0.797


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(1000, 500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.71      0.54      0.61       519
    negative       0.61      0.77      0.68       481

    accuracy                           0.65      1000
   macro avg       0.66      0.65      0.64      1000
weighted avg       0.66      0.65      0.64      1000

Matrix: Confusion
 [[279 240]
 [113 368]]
Accuracy:
 0.647


## TD-IDF

In [30]:
vectorizer = TfidfVectorizer(max_features=2000)
X_train_idf = vectorizer.fit_transform(X_train)
X_test_idf = vectorizer.transform(X_test)

In [31]:
tf_idf_f1, tf_idf_mlp = eval_mlp(X_train_idf, X_test_idf, Y_train, Y_test)

/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.66      0.34      0.45       519
    negative       0.53      0.81      0.64       481

    accuracy                           0.57      1000
   macro avg       0.60      0.58      0.55      1000
weighted avg       0.60      0.57      0.54      1000

Matrix: Confusion
 [[176 343]
 [ 89 392]]
Accuracy:
 0.568


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.56      0.67      0.61       519
    negative       0.55      0.43      0.49       481

    accuracy                           0.56      1000
   macro avg       0.56      0.55      0.55      1000
weighted avg       0.56      0.56      0.55      1000

Matrix: Confusion
 [[348 171]
 [272 209]]
Accuracy:
 0.557


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(500, 250, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.55      0.63      0.59       519
    negative       0.53      0.45      0.49       481

    accuracy                           0.54      1000
   macro avg       0.54      0.54      0.54      1000
weighted avg       0.54      0.54      0.54      1000

Matrix: Confusion
 [[327 192]
 [265 216]]
Accuracy:
 0.543


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model config: hidden_layer_sizes=(1000, 500, 250), activation=tanh
Report: Classification
               precision    recall  f1-score   support

    positive       0.61      0.33      0.43       519
    negative       0.52      0.77      0.62       481

    accuracy                           0.54      1000
   macro avg       0.56      0.55      0.53      1000
weighted avg       0.57      0.54      0.52      1000

Matrix: Confusion
 [[172 347]
 [109 372]]
Accuracy:
 0.544
Model config: hidden_layer_sizes=(1000, 500, 250), activation=relu
Report: Classification
               precision    recall  f1-score   support

    positive       0.00      0.00      0.00       519
    negative       0.48      1.00      0.65       481

    accuracy                           0.48      1000
   macro avg       0.24      0.50      0.32      1000
weighted avg       0.23      0.48      0.31      1000

Matrix: Confusion
 [[  0 519]
 [  0 481]]
Accuracy:
 0.481


/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mryf/PycharmProjects/ml_final_project/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mryf/Pych

## Comparison

In [32]:
print('Best scores:')
print(f'W2V: {w2v_f1} with params: {w2v_mlp.get_params()}')
print(f'BoW: {bow_f1} with params: {bow_mlp.get_params()}')
print(f'Tf-Idf: {tf_idf_f1} with params: {tf_idf_mlp.get_params()}')

idx = np.argmax([w2v_f1, bow_f1, tf_idf_f1])
best_mlp = [w2v_mlp, bow_mlp, tf_idf_mlp][idx]
with open('best.pkl', 'wb') as f:
    pickle.dump(best_mlp, f)


Best scores:
W2V: 0.8353658536585367 with params: {'activation': 'tanh', 'alpha': 1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (1000, 500, 250), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 10, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
BoW: 0.8 with params: {'activation': 'tanh', 'alpha': 1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (1000, 500, 250), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 10, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction'